In [1]:

import os,subprocess,sys,shutil
import pandas as pd
print(os.path.abspath(''))

from subprocess import run, PIPE, STDOUT

try:
    sys.path.append(os.path.join(os.path.abspath(''), '..', '..', '..', '..', "tools")) # tutorial in tests
    sys.path.append(os.path.join(os.path.abspath(''), '..', '..', '..', '..', "tools","xml")) # xml converter
    sys.path.append(os.path.join(os.environ.get("SUMO_HOME", os.path.join(os.path.abspath(''), "..", "..", "..")), "tools")) # tutorial in docs
    from sumolib import checkBinary
except ImportError:    
    sys.exit("please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")

netconvertBinary = checkBinary('netconvert')
sumoBinary = checkBinary('sumo')


/Users/pavelmyznikov/PycharmProjects/OD-predict


In [2]:
# build network from osm file
osm_file = 'small_london.osm'
retcode = subprocess.Popen([netconvertBinary, "--osm-files", f"data/{osm_file}","-o","data/my_net.net.xml"],stdout=PIPE,stderr=PIPE)
for msg in retcode.communicate(): print(msg.decode('utf-8'), end='\n\n') 
try: shutil.copy("data/my_net.net.xml", "net.net.xml")
except: print("Missing 'my_net.net.xml'")
print(">> Netbuilding closed with status %s" % retcode.returncode)


Success.




>> Netbuilding closed with status 0


In [18]:
simulations_number = 1000

In [19]:
from tqdm.notebook import trange, tqdm

In [20]:
subprocess.Popen(['ulimit', '-n', '1000'])

In [21]:
times="4000"

In [22]:
dfs = []
simulation_last = 721
for i in trange(simulation_last, simulations_number):
    # add random trips
    retcode = subprocess.Popen(["python",'tools/randomTrips.py', "-n", "data/my_net.net.xml","-r","data/routes.rou.xml", "-e", times, "-o", "simulation_data/trips/trips{}.xml".format(str(i)), "-p", "10", "-l"], stdout=PIPE,stderr=PIPE,close_fds=True)
    for msg in retcode.communicate(): print(msg.decode('utf-8')) 

    retcode = subprocess.Popen([sumoBinary, "-c", "data/london.sumocfg","--no-step-log", "-r","data/routes.rou.xml", "--additional-files","data/my_output.out.xml", "--end", times], stdout=PIPE,stderr=PIPE,close_fds=True)
    for msg in retcode.communicate(): print(msg.decode('utf-8')) 

    # convert xml to csv
    retcode = subprocess.Popen(['python','tools/xml/xml2csv.py','-s',',','data/edge_data.xml','--output',"simulation_data/edges/edges{}.csv".format(str(i))],stdout=PIPE,stderr=PIPE,close_fds=True)
    for msg in retcode.communicate(): print(msg.decode('utf-8')) 
#     print(">> xml to  csv conversion closed with status %s" % retcode.returncode)
    
#     dfs.append(pd.read_csv('data/edge_data.csv'))

Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips721.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips722.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips723.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips724.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... do

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips741.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips742.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips743.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips744.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips758.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips759.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips760.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips761.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips778.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips779.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips780.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips781.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips803.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips804.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips805.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips806.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

...

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips913.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips914.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips915.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips916.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro



Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips931.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips932.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips933.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips934.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... 

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips956.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips957.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips958.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips959.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips972.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips973.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips974.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips975.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips992.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips993.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips994.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.rou.xml


Loading configuration ... done.




Success.
calling  /usr/local/opt/sumo/share/sumo/bin/duarouter -n data/my_net.net.xml -r simulation_data/trips/trips995.xml --ignore-errors --begin 0 --end 4000.0 --no-step-log --no-warnings -o data/routes.ro

In [3]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET 

net_file = 'data/my_net.net.xml'


class GridNet():
    # take in a network file an returns a df with junctions grouped by a reactangular grid
    # inputs:
    #   net_file - a sumo net.xml file generated by netconverter 
    #   grid     - specifications of regcatngular grid (x,y) default=(32,32)  

    def __init__(self, net_file, grid=(32,32)):
        
        self.net_file = net_file
        self.grid = grid
        
        # parse network file in element tree
        self.root = ET.parse(self.net_file).getroot()

    def get_junctions(self):
        # return a df of junctions with x and y locations 
        junctions = pd.DataFrame(columns =['x','y', 'id'])
        i = 1
        for child in self.root:
            if child.tag == 'junction':
                junctions.loc[i] = pd.Series({'x':float(child.attrib['x']),
                                              'y':float(child.attrib['y']), 
                                              'id':str(child.attrib['id'])})
                i+=1
        return junctions
    
    def get_edges(self):
        edges = pd.DataFrame(columns=['edge_id', 'junction_id'])
        i = 1
        for child in self.root:
            if child.tag == 'edge' and 'from' in child.attrib:
                edges.loc[i] = pd.Series({'edge_id': str(child.attrib['id']),
                                         'junction_id': str(child.attrib['from'])
                                         })
                i += 1
        return edges

    def min_max_xy(self, junctions):
        # return minimimum and maximums of x and y locations
        stats = junctions.describe()
        return stats['x']['min'], stats['x']['max'], stats['y']['min'], stats['y']['max']

    def grid_net(self, junctions):
        # get seperating sets (endpoints)
        minX, maxX, minY, maxY = self.min_max_xy(junctions)
        x_sep = np.linspace(minX, maxX, num=self.grid[0])
        y_sep = np.linspace(minY, maxY, num=self.grid[1])
        return x_sep, y_sep

    def group_junctions(self):
        junctions = self.get_junctions()
        junctions['x'] = pd.cut(junctions['x'], self.grid[0], right=False, labels=range(self.grid[0]))
        junctions['y'] = pd.cut(junctions['y'], self.grid[1], right=False, labels=range(self.grid[1]))
        return junctions

    def map_edges_coords(self):
        edges = self.get_edges()
        junctions = self.group_junctions()
        return edges.merge(junctions, left_on='junction_id', right_on='id')[['edge_id', 'x', 'y']]



In [4]:
net_file = 'data/my_net.net.xml'
gn = GridNet(net_file)

In [5]:
edges_coords = gn.map_edges_coords()

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [24]:
from glob import glob
import re

In [25]:
flows_filenames = glob('simulation_data/edges/*.csv')
flows_dataframes = []
for i, flows_file in enumerate(flows_filenames):
    sim_id = int(re.findall(r'edges(\d+)', flows_file)[0])
    flow_dataframe = pd.read_csv(flows_file)
    flow_dataframe['sim_id'] = sim_id
    flows_dataframes.append(flow_dataframe)

flows = pd.concat(flows_dataframes)

In [26]:
flows = flows.merge(edges_coords)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


From documentation:

The following <edgeData> values are all reasonable counts in that they count each vehicle only once but they include/exclude some special cases.

- entered
- entered + departed
- left
- left + arrived

In [27]:
from sklearn import preprocessing

In [28]:
le_times = preprocessing.LabelEncoder()
le_times.fit(flows['interval_begin'].unique())
flows['time'] = le_times.transform(flows['interval_begin'])

In [29]:
flows['vehicles_number'] = flows['edge_departed'] + flows['edge_entered']

In [30]:
flows[['sim_id', 'time', 'x', 'y', 'vehicles_number']].to_csv('flows.csv', index=False)

In [42]:
flows.tail()

,edge_arrived,edge_density,edge_departed,edge_entered,edge_id,edge_laneChangedFrom,edge_laneChangedTo,edge_left,edge_occupancy,edge_overlapTraveltime,...,edge_traveltime,edge_waitingTime,interval_begin,interval_end,interval_id,sim_id,x,y,time,vehicles_number
68316648,0,NaN,0,0,751842924#2,0,0,0,NaN,NaN,...,NaN,NaN,3000.0,3200.0,d1,308,7,11,15,0
68316649,0,NaN,0,0,751842924#2,0,0,0,NaN,NaN,...,NaN,NaN,3200.0,3400.0,d1,308,7,11,16,0
68316650,0,NaN,0,0,751842924#2,0,0,0,NaN,NaN,...,NaN,NaN,3400.0,3600.0,d1,308,7,11,17,0
68316651,0,NaN,0,0,751842924#2,0,0,0,NaN,NaN,...,NaN,NaN,3600.0,3800.0,d1,308,7,11,18,0
68316652,0,NaN,0,0,751842924#2,0,0,0,NaN,NaN,...,NaN,NaN,3800.0,4000.0,d1,308,7,11,19,0


In [6]:
import xml.etree.ElementTree as ET

In [31]:
trips_path = 'data/trips.trips.xml'

In [32]:
trips_filenames = glob('simulation_data/trips/*.xml')
trips_data = []

for i, trips_path in enumerate(trips_filenames):
    sim_id = int(re.findall(r'trips(\d+)', trips_path)[0])
    trips_xml = ET.parse(trips_path) 
    root = trips_xml.getroot()
    for trip in root.iter('trip'):
        trips_data.append([sim_id, trip.attrib['id'], float(trip.attrib['depart']), trip.attrib['from'], trip.attrib['to']])

In [33]:
trips = pd.DataFrame(trips_data, columns=['sim_id', 'id', 'depart', 'from', 'to'])

In [34]:
trips = trips.merge(edges_coords, left_on='from', right_on='edge_id', suffixes=('_trips', '_from'))

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [35]:
trips = trips.merge(edges_coords, left_on='to', right_on='edge_id', suffixes=('_trips', '_to'))

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [36]:
trips.rename(columns={"x_trips": "x_from", "y_trips": "y_from"}, inplace=True)

In [37]:
trips.head()

,sim_id,id,depart,from,to,edge_id_trips,x_from,y_from,edge_id_to,x_to,y_to
0,434,0,0.0,466645914#0,26764841,466645914#0,8,0,26764841,3,14
1,583,155,1550.0,466645914#0,26764841,466645914#0,8,0,26764841,3,14
2,224,230,2300.0,466645914#0,26764841,466645914#0,8,0,26764841,3,14
3,224,352,3520.0,466645914#0,26764841,466645914#0,8,0,26764841,3,14
4,579,103,1030.0,466645914#0,26764841,466645914#0,8,0,26764841,3,14


In [38]:
trips['cell_from'] =  trips['x_from'].astype(str).str.cat(trips['y_from'].astype(str), sep="_")
trips['cell_to'] =  trips['x_to'].astype(str).str.cat(trips['y_to'].astype(str), sep="_")

In [ ]:
trips['sim_id', 'id'].to_csv('trips.csv', index=False)

In [40]:
le_cells = preprocessing.LabelEncoder()
le_cells.fit(np.unique(trips[['cell_from', 'cell_to']].values))
trips['cell_from'] = le_times.transform(trips['cell_from'])
trips['cell_to'] = le_times.transform(trips['cell_to'])

ValueError: y contains previously unseen labels: '8_0'

In [17]:
import numpy as np

In [31]:
le_edges = preprocessing.LabelEncoder()
le_edges.fit(np.unique(trips[['from', 'to']].values))

LabelEncoder()

In [41]:
trips['time'] = le_times.transform(np.floor(trips['depart'] / 200) * 200)
# trips['from'] = le_edges.transform(trips['from'])
# trips['to'] = le_edges.transform(trips['to'])

ValueError: y contains previously unseen labels: [500.0, 1500.0, 2500.0, 3500.0]

In [35]:
trips.head()

,sim_id,id,depart,from,to,edge_id_trips,x_from,y_from,edge_id_to,x_to,y_to,cell_from,cell_to,time
0,0,0,0.0,27695728#2,466431903#1,27695728#2,11,6,466431903#1,4,6,11_6,4_6,0
1,337,824,824.0,27695728#2,466431903#1,27695728#2,11,6,466431903#1,4,6,11_6,4_6,1
2,464,990,990.0,27695728#2,466431903#1,27695728#2,11,6,466431903#1,4,6,11_6,4_6,1
3,3,790,790.0,26445205#0,466431903#1,26445205#0,3,14,466431903#1,4,6,3_14,4_6,1
4,282,26,26.0,26445205#0,466431903#1,26445205#0,3,14,466431903#1,4,6,3_14,4_6,0


In [36]:
trips[['sim_id', 'time', 'x_from', 'y_from', 'x_to', 'y_to']].to_csv('trips.csv', index=False)

In [29]:
trips[['from','to']].values.max()

3362

In [26]:
import tensorflow as tf

In [31]:
trips_matrix = tf.SparseTensor(
    indices=trips[['time','from','to']].values, 
    values=np.ones(trips.shape[0]), 
    dense_shape=[trips['time'].max()+1, trips[['from','to']].values.max(), trips[['from','to']].values.max()]
)

# sparse.coo_matrix(
#     (np.ones(trips.shape[0]),
#      (
#          trips['time'].values,
#          trips['from'].values,
#          trips['to'].values
#      )
#     ), 
#     shape=(trips['time'].max()+1, trips['from'].max()+1, trips['to'].max()+1 )
#   )

In [16]:
# load data and apply some transformations to get sparse speed avg speed matrix
import numpy as np

df = pd.read_csv('results.csv')

# drop node and NANs
node_index = df[df['edge_id'].str.contains(':',na = True )].index
df.drop(node_index, inplace=True)
#df.dropna(how='all',inplace=True)

avg_speed = df[['data_timestep','edge_id','lane_meanspeed']].groupby(['data_timestep','edge_id']).agg('mean')

print(avg_speed.head(20))

max_time = max(df['data_timestep'])
bins = np.arange(0,max_time, 20)

avg_speed_x20 = df[['data_timestep','edge_id','lane_meanspeed']].groupby([pd.cut(df['data_timestep'], bins),'edge_id']).agg('mean')
print(avg_speed_x20.head(20))

C:\Users\crm0376\.local\conda_root\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,11,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: 